In [1]:
#read the reviews
import pandas as pd
df1 = pd.read_csv(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\TopicModellingDataset\scrapeDataset.csv")
df2= pd.read_csv(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\TopicModellingDataset\scrapeDataset2.csv")

In [2]:
print(df1.shape)
print(df2.shape)

(43840, 3)
(8420, 3)


In [3]:
df = df1.merge(df2, how='outer')
print(df[43839:])
print(df.shape)

       Unnamed: 0                                            product  \
43839       43839  Epson - EcoTank ET-2850 All-in-One Supertank I...   
43840           0  Canon - PIXMA MG3620 Wireless All-In-One Inkje...   
43841           1  Canon - PIXMA MG3620 Wireless All-In-One Inkje...   
43842           2  Canon - PIXMA MG3620 Wireless All-In-One Inkje...   
43843           3  Canon - PIXMA MG3620 Wireless All-In-One Inkje...   
...           ...                                                ...   
52255        8415  HP - DeskJet 2755e Wireless Inkjet Printer wit...   
52256        8416  HP - DeskJet 2755e Wireless Inkjet Printer wit...   
52257        8417  HP - DeskJet 2755e Wireless Inkjet Printer wit...   
52258        8418  HP - DeskJet 2755e Wireless Inkjet Printer wit...   
52259        8419  HP - DeskJet 2755e Wireless Inkjet Printer wit...   

                                                  review  
43839  Print quality is great, and filling the ink ta...  
43840  Red is my 

In [4]:
print(df['product'].unique())
print("\n\nNumber of products: ", len(df['product'].unique()))

['HP - OfficeJet Pro 9015e Wireless All-In-One Inkjet Printer with 6 months of Instant Ink Included with HP+ - White'
 'Epson - WorkForce Pro WF-3820 Wireless All-in-One Printer'
 'HP - DeskJet 2734e Wireless All-In-One Inkjet Printer with 3 months of Instant Ink included from HP+ - White'
 'Brother - HL-L2395DW Wireless Black-and-White All-In-One Refresh Subscription Eligible Laser Printer - Gray'
 'Epson - EcoTank ET-2800 Wireless All-in-One Supertank Inkjet Printer - White'
 'Brother - HL-L2370DW Wireless Black-and-White Refresh Subscription Eligible Laser Printer - Gray'
 'HP - OfficeJet Pro 8025e Wireless All-In-One Inkjet Printer with 6 months of Instant Ink Included with HP+ - White'
 'Epson - Expression Premium XP-7100 Wireless All-In-One Inkjet Printer - Black'
 'HP - ENVY Inspire 7955e Wireless All-In-One Inkjet Photo Printer with 3 months of Instant Ink included with HP+ - White & Sandstone'
 'Brother - MFC-L2710DW Wireless Black-and-White All-in-One Refresh Subscription Eli

In [5]:
df.dtypes

Unnamed: 0     int64
product       object
review        object
dtype: object

In [6]:
new_df = pd.DataFrame()

In [7]:

import nltk
nltk.download('punkt')
#Sentence Splitting (each sentence is now its own review)
from nltk.tokenize import sent_tokenize
from cleantext import clean

product = []
review = []

for index in df.index:
    sentences = sent_tokenize(df.loc[index, 'review'])
    for sentence in sentences:
        product.append(df.loc[index, 'product'])
        review.append(clean(sentence, no_emoji=True))

# append columns to new_df
new_df['product'] = product
new_df['review'] = review



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ethan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
new_df.head(5)

,product,review
0,HP - OfficeJet Pro 9015e Wireless All-In-One I...,after having my last hp printer for 12 beautif...
1,HP - OfficeJet Pro 9015e Wireless All-In-One I...,so far so good.
2,HP - OfficeJet Pro 9015e Wireless All-In-One I...,i've had the printer for about a month and no ...
3,HP - OfficeJet Pro 9015e Wireless All-In-One I...,"works great, setup was quick and easy."
4,HP - OfficeJet Pro 9015e Wireless All-In-One I...,i'm truly satisfied with my recent purchase.


In [9]:
print(len(new_df))

158166


In [10]:
#Text Preprocessing for aspect extraction
import string
import emoji
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
# spacy for lemmatization
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):

    #1. Generating the list of words in the tweet (hastags and other punctuations removed)
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)

    #2. clean the number
    text = re.sub(r'[0-9]', '', text)

    #3. lower the text
    text = text.lower()

    #4. conver the emoji to text form
    text = emoji.demojize(text)

    #5. remove punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')

    #6. tokenize the text
    text = word_tokenize(text)

    #7. remove empty token
    text = [t for t in text if len(t) > 0]

    #8. remove non-alphabetical token
    text = [t for t in text if t.isalpha()]

    #9. replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)

    #10. remove the stopwords
    text = [i for i in text if i not in stopwords]

    #11. stem the text
    #porter_stemmer = PorterStemmer()
    #text = [porter_stemmer.stem(w) for w in text]

    #11. lemmatize the text
    text = lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return text


def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(sent))
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ethan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
print(new_df.iloc[1,1])
print(type(new_df.iloc[1,1]))
print(preprocess( new_df.iloc[1,1]))

so far so good.
<class 'str'>
['far', 'good']


In [ ]:
#Preprocess the input data
nltk.download('wordnet')
cleaned_text = []
counter = 1

# Don't print Preprocessing row because PDF had issues when trying to export it (had over 2000 pages of otuput due to the preprocessing line)
for text in new_df["review"]:
    # print(f'Preprocessing row {counter}')
    counter+=1
    cleaned_text.append(preprocess(str(text)))

new_df["Clean"] = cleaned_text
new_df = new_df.dropna()



In [13]:
new_df.to_csv(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\preprocessed_dataLDA.csv")
print('Preprocessing done.')

Preprocessing done.


In [14]:
print(type(cleaned_text))

<class 'list'>


In [15]:
#Extract 100k from original dataset for BERTopic, skip this if the dataset is < 100k
#import pandas as pd
#pd.read_excel(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Lab 6\preprocessed_data.xlsx").sample(n=100000).to_excel(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Lab 6\extracted_data.xlsx", index=False)

In [17]:
import pandas as pd
#extracted_data = pd.read_excel("D:/research/data/reviews/extracted_data.xlsx")
extracted_data = pd.read_csv(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\preprocessed_dataLDA.csv")
extracted_data.head()

,Unnamed: 0,product,review,Clean
0,0,HP - OfficeJet Pro 9015e Wireless All-In-One I...,after having my last hp printer for 12 beautif...,"['last', 'printer', 'beautiful', 'year', 'time..."
1,1,HP - OfficeJet Pro 9015e Wireless All-In-One I...,so far so good.,"['far', 'good']"
2,2,HP - OfficeJet Pro 9015e Wireless All-In-One I...,i've had the printer for about a month and no ...,"['printer', 'month', 'complaint']"
3,3,HP - OfficeJet Pro 9015e Wireless All-In-One I...,"works great, setup was quick and easy.","['work', 'great', 'setup', 'quick', 'easy']"
4,4,HP - OfficeJet Pro 9015e Wireless All-In-One I...,i'm truly satisfied with my recent purchase.,"['truly', 'satisfy', 'recent', 'purchase']"


In [18]:
#LDA model training

import gensim.corpora as corpora
import gensim.models as models

from ast import literal_eval
from pprint import pprint

data_words = []
for x in extracted_data['Clean']:
    data_words.append(literal_eval(x))

# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# number of topics
num_topics = 50
num_words = 10

# Build LDA model
lda_model1 =models.LdaMulticore(corpus=corpus,
                              id2word=id2word,
                              num_topics=num_topics)



In [19]:
#save LDA Model
import pickle
topics = lda_model1.print_topics(num_topics=num_topics, num_words=num_words)
topic_list = []
for topic in topics:
  topic_list.append(topic[1])

#df['topics1'] = topic_list1
df_topics = pd.DataFrame(topic_list,  columns =['topics'])

df_topics.to_csv(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_topics.csv")
pickle.dump(lda_model1, open(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_model.model", 'wb'))#save lda model
pickle.dump(id2word, open(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_model.dict", 'wb'))#save dictionary
pickle.dump(corpus, open(r"C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_model.corpus", 'wb'))#save corpus



In [20]:
pprint(lda_model1.print_topics())
doc_lda = lda_model1[corpus]

[(18,
  '0.084*"printer" + 0.045*"easy" + 0.032*"print" + 0.031*"ink" + 0.030*"good" '
  '+ 0.025*"buy" + 0.024*"great" + 0.023*"set" + 0.017*"cartridge" + '
  '0.015*"fast"'),
 (4,
  '0.066*"printer" + 0.029*"great" + 0.024*"use" + 0.022*"print" + '
  '0.018*"small" + 0.018*"work" + 0.017*"far" + 0.016*"easy" + 0.014*"buy" + '
  '0.014*"good"'),
 (42,
  '0.061*"printer" + 0.027*"ink" + 0.023*"use" + 0.020*"print" + 0.018*"buy" + '
  '0.018*"work" + 0.018*"last" + 0.016*"good" + 0.015*"year" + 0.013*"love"'),
 (43,
  '0.059*"printer" + 0.030*"print" + 0.028*"ink" + 0.023*"month" + '
  '0.019*"great" + 0.016*"work" + 0.014*"old" + 0.012*"set" + 0.012*"setup" + '
  '0.010*"get"'),
 (7,
  '0.105*"printer" + 0.044*"great" + 0.029*"use" + 0.023*"print" + '
  '0.020*"work" + 0.014*"ink" + 0.013*"easy" + 0.013*"home" + 0.013*"laser" + '
  '0.011*"problem"'),
 (1,
  '0.050*"set" + 0.048*"easy" + 0.043*"print" + 0.038*"printer" + 0.031*"use" '
  '+ 0.022*"phone" + 0.016*"great" + 0.012*"get" + 

In [21]:
import pickle

#lda_model_file =
loaded_lda_model = pickle.load(open(r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_model.model' , 'rb')) #load trained model

loaded_dictionary = pickle.load(open(r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_model.dict', 'rb')) #load dictionary
loaded_corpus = pickle.load(open(r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\LDA\LDA_model.corpus', 'rb')) # load corpus


In [22]:

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(loaded_lda_model, loaded_corpus, loaded_dictionary)


In [23]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17     0.077157  0.038002       1        1  4.270691
28     0.073144  0.035629       2        1  3.460051
34    -0.042514  0.057110       3        1  3.409871
1     -0.024862  0.041776       4        1  3.144798
15    -0.001447  0.119368       5        1  3.144745
32    -0.067300  0.085715       6        1  2.926295
35     0.110040 -0.023672       7        1  2.772535
3     -0.082071 -0.001529       8        1  2.529203
11    -0.091571 -0.030174       9        1  2.365469
33     0.023303 -0.082955      10        1  2.351796
38     0.015101 -0.053974      11        1  2.203175
25    -0.025648 -0.041915      12        1  2.133176
43     0.020642 -0.042416      13        1  2.128406
37    -0.058207 -0.019820      14        1  2.091844
30    -0.065438 -0.015283      15        1  2.080811
22     0.049592  0.047570      16        1  2.077617
41    -0.048024 -0.011772      17        1  2.037459
48     0.015645 -0.026704      18        1  2.002922
12    -0.027674 -0.019169      19        1  1.971023
31    -0.027147  0.006191      20        1  1.969424
9      0.048178  0.001927      21        1  1.954149
8     -0.006882  0.011754      22        1  1.907766
4      0.011892 -0.006229      23        1  1.870593
14    -0.032722 -0.042948      24        1  1.868719
7      0.056040 -0.028608      25        1  1.853850
42     0.039793 -0.021737      26        1  1.851252
40    -0.041838 -0.003332      27        1  1.847029
24    -0.009431 -0.030437      28        1  1.817305
36     0.008134  0.097695      29        1  1.812648
10    -0.037615 -0.019974      30        1  1.798573
26     0.002123 -0.015420      31        1  1.797648
5      0.019731 -0.013244      32        1  1.765449
47    -0.002772 -0.066561      33        1  1.761284
19     0.012495 -0.019223      34        1  1.736867
45     0.013088 -0.001138      35        1  1.669392
13    -0.050555  0.015545      36        1  1.629953
2      0.029764 -0.001907      37        1  1.614144
0     -0.016719  0.011696      38        1  1.581601
6      0.035969 -0.012476      39        1  1.569350
49     0.028517  0.017717      40        1  1.565511
44     0.005097 -0.005956      41        1  1.475776
21     0.035067  0.047184      42        1  1.444752
46    -0.020720  0.040918      43        1  1.436185
23    -0.000455 -0.022181      44        1  1.412506
16     0.005000 -0.019004      45        1  1.404581
27     0.005955 -0.009011      46        1  1.387875
39     0.001182 -0.001909      47        1  1.357088
20    -0.002023 -0.015285      48        1  1.317642
29    -0.001868  0.026955      49        1  1.249800
18     0.042855  0.023211      50        1  1.169404, topic_info=         Term          Freq         Total Category  logprob  loglift
4     printer  44155.000000  44155.000000  Default  30.0000  30.0000
50      print  27399.000000  27399.000000  Default  29.0000  29.0000
12       easy  16707.000000  16707.000000  Default  28.0000  28.0000
13      great  13547.000000  13547.000000  Default  27.0000  27.0000
107       use  16995.000000  16995.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
42      price     54.608953   5697.722790  Topic50  -5.1569  -0.1989
104     paper     51.447018   5438.105405  Topic50  -5.2165  -0.2120
201  printing     46.616106   5021.659887  Topic50  -5.3151  -0.2309
23       home     44.171410   4647.119461  Topic50  -5.3690  -0.2072
377      wifi     41.823723   3838.083629  Topic50  -5.4236  -0.0706

[4102 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
157      33  0.663014  abandon
69        1  0.019150     able
69        2  0.015009     able
69        3  0.042440     able
69        4  0.129390     able
...     ...       ...      ...
3644     49  0.018971  youtube
3813     21  0.585694      yse
2599     22  0.116175    